# Исследование надежности заемщиков

## Ⅰ. Изучим общую информацию о данных.

In [1]:
# импортируем библиотеку pandas и прочитаем csv-файл
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    print('Ошибка чтения данных')

In [2]:
#  выведем первые 20 строк датафрейма
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# выведем основную информацию о датафрейме
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Ⅱ. Предобработка данных.

### Удаление пропусков

In [4]:
# выведем пропущенные значения для каждого стлбца
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах.   
Сначала разберёмся с `total_income`. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [5]:
for job in data['income_type'].unique():
    job_median = data.loc[data['income_type']==job]['total_income'].median()
    data.loc[(data['income_type']==job)&(data['total_income'].isna()==True),'total_income'] = job_median

### Обработка аномальных значений

В столбце `days_employed` присутствуют артефакты, в данном случае это отрицательное количество дней трудового стажа. Заменим все отрицательные значения положительными с помощью метода `abs()`.

In [6]:
data['days_employed'] = abs(data['days_employed'])

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие цифры сложно, поэтому оставим их как есть.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [11]:
for job in data['income_type'].unique():
    days_median = data.loc[data['income_type']==job]['days_employed'].median()
    data.loc[(data['income_type'] == job) & (data['days_employed'].isna()),'days_employed'] = days_median

In [12]:
# убедимся, что все пропуски заполнены
data['days_employed'].isna().sum()

0

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [13]:
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
# выведем на экран количество строк-дубликатов в данных.
data.duplicated().sum()

71

In [16]:
# удалим дубликаты
data = data.drop_duplicates()
data.duplicated().sum()

0

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
# создадим функцию categorize_income()
def categorize_income(income):
    if income > 1000000:
        return 'A'
    elif income > 200000:
        return 'B'
    elif income > 50000:
        return 'C'
    elif income > 30000:
        return 'D'
    else:
        return 'E'

In [18]:
# применим функцию методом apply()
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
# создадим функцию categorize_purpose()
def categorize_purpose(deal):
    if deal.find('автомобил')>=0:
        return 'операции с автомобилем'
    elif deal.find('жиль')>=0:
        return 'операции с недвижимостью'
    elif deal.find('недвижимост')>=0:
        return 'операции с недвижимостью'
    elif deal.find('свадьб')>=0:
        return 'проведение свадьбы'
    elif deal.find('образовани')>=0:
        return 'получение образования'

In [21]:
# применим функцию методом apply()
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [28]:
# выведем первые 10 строк изменённого датафрейма
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


## Ⅲ. Исследуем данные и ответим на вопросы.

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Чтобы найти зависимость между количеством детей у заёмщика и возвратом кредита в срок необходимо найти среднее по долгу с учётом группировки по количеству имеющихся детей. Далее, на основании этих данных построим сводную таблицу.

In [23]:
table1 = (data.groupby('children')['debt'].agg(['sum','count','mean'])).sort_values('mean')
table1['mean'] = table1['mean']*100
table1

,sum,count,mean
children,,,
5,0,9,0.000000
0,1063,14091,7.543822
3,27,330,8.181818
1,444,4808,9.234609
2,194,2052,9.454191
4,4,41,9.756098


Колонка `children` показывает количество детей у заёмщика,  
`debt` - количество заёмщиков невозвративших кредит в срок,  
`debt_count` - количество заёмщиков в каждой группе по кол-ву детей,  
`debt_ratio` - отношение `debt` к `debt_count`.  

Сразу обращает на себя внимание отсутствие невозвратного кредита у заёмщиков с количеством детей 5 человек. У категории бездетных самый низкий уровень задолженностей.

Также, отличительной особенностью является уменьшение количества займов `debt_count` с ростом числа детей.

In [24]:
table1.sort_values('count', ascending=False)

,sum,count,mean
children,,,
0,1063,14091,7.543822
1,444,4808,9.234609
2,194,2052,9.454191
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


>**Вывод:**  
>Показательно, что заёмщиков с пятью детьми всего девять и это не может являться достаточным основанием чтобы делать обоснованные выводы для этой категории. Несмотря на этот факт, наблюдается линейная зависимость между задолженностью и количеством детей.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Найдём отношение невозвративших вовремя кредит заёмщиков к общему числу заёмщиков с учётом семейного положения и оформим результат в виде сводной таблицы.

In [25]:
table2 = (data.groupby('family_status')['debt'].agg(['sum','count','mean'])).sort_values('mean')
table2['mean'] = table2['mean']*100
table2['age']=data.groupby('family_status')['dob_years'].median()
table2

,sum,count,mean,age
family_status,,,,
вдовец / вдова,63,951,6.624606,58
в разводе,84,1189,7.064760,46
женат / замужем,927,12261,7.560558,43
гражданский брак,385,4134,9.313014,41
Не женат / не замужем,273,2796,9.763948,36


>**Вывод:**  
>Индивиды из категорий вдовцы/вдовы и в разводе представляет незначительную долю в совкупности, хотя и имеют самый низкий процент невозвратов. Больше всего невозврата у неженатых/незамужних и находящихся в гражданском браке. Можно предположить, что это является следствие возраста заёмщиков, потому что такие категории как гражданский брак и Не женат / не замужем образуют самые молодые заёмщики среди остальных, склонных пероценивать свои возможности.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Построим сводную таблицу показывающую зависимость между уровнем дохода и возвратом кредита в срок в соответствии со следующими категориями.

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [26]:
table3 = (data.groupby('total_income_category')['debt'].agg(['sum','count','mean'])).sort_values('mean')
table3['mean'] = table3['mean']*100
table3

,sum,count,mean
total_income_category,,,
D,21,349,6.017192
B,354,5014,7.060231
A,2,25,8.000000
C,1353,15921,8.498210
E,2,22,9.090909


По количеству представителей выделяются две категории заёмщиков `B` и `C`, причём последняя в три раза больше, это индивиды со средним уровнем дохода. Самая высокая доля невозвратов среди людей с низким доходом, но их и представлено меньше остальных. 

>**Вывод:**  
>если не учитывать фактор количества, то видна незначительная тенденция роста невозврата кредитов с уменьшением уровня доходов.

#### Как разные цели кредита влияют на его возврат в срок?

Построим сводную таблицу показывающую зависимость между целью кредита и возвратом кредита в срок в соответствии со следующими категориями:

- `операции с автомобилем`,
- `операции с недвижимостью`,
- `проведение свадьбы`,
- `получение образования`.

In [27]:
table4 = (data.groupby('purpose_category')['debt'].agg(['sum','count','mean'])).sort_values('mean')
table4['mean'] = table4['mean']*100
table4['age']=data.groupby('purpose_category')['dob_years'].mean().astype('int')
table4

,sum,count,mean,age
purpose_category,,,,
операции с недвижимостью,780,10751,7.255139,43
проведение свадьбы,183,2313,7.911803,43
получение образования,369,3988,9.252758,43
операции с автомобилем,400,4279,9.347978,43


>**Вывод:**  
>`операции с недвижимостью` является категорией с самым низким уровнем невозврата кредита в срок. Это может быть обусловленно значительными суммами сделок с недвижимостью и, соответственно, более ответственным подходом к операциям. Самая высокая доля невозвратов у категорий `получение образования` и `операции с автомобилем`. Можно утверждать, что сделки с автомобилями являются самыми рискованными видом кредита. Ещё одная замечательная особенность, что средний возраст всех заёмщиков по данным категориям составляет 43 года.

## Ⅳ. Общий вывод.

Предоставленные данные были получены в текстовом формате и требовали предварительной обработки для дальнейшего анализа. В процессе предобработки было обнаружено, что некоторые значения в данных о трудовом стаже и ежемесячном доходе отсутствуют. Для заполнения этих пропусков были использованы медианные значения соответствующих категорий. Также было обнаружено, что в данных о ежемесячных доходах имеются артефакты в виде отрицательных значений, которые были исправлены. Для обеспечения точности и надежности данных, были удалены записи о количестве детей с отрицательными и аномально высокими значениями. Кроме того, в данных были обнаружены дубликаты, которые были удалены, чтобы избежать искажения результатов анализа. Для удобства представления информации и более эффективного анализа, мы провели категоризацию двух показателей: уровень дохода и цель кредита. Это позволило нам группировать данные по определенным критериям и проводить сравнительный анализ между различными группами.

На этапе анализа данных выявлены следующие закономерности:

- при увеличении количества детей кредитоспособность заёмщика падает;
- одинокие молодые люди моложе 43 лет и находящиеся в гражданском браке показали высокие доли невозврата кредита, в то время как овдовевшие супруги и лица в разводе самые низкие;
- невысокий уровень дохода является причиной задержек выплат;
- такие целевые категории как 'получение образования' и 'операции с автомобилем' составляют наиболее рискованные области кредитования, 'операции с недвижимостью' показали самый низкий риск для кредитования

## Ⅴ. Замечания и рекомендации.

Количество заёмщиков по каждой из категорий, довольно сильно различаются. Например, в категории `количество детей`, лиц имеющих 5 детей всего 9 человек и судить по этим данным о достоверности полученных результатов сложно. А в категории `общий доход` заёмщики с самым высоким и самым низким доходами составляют 25 и 22 человека соответственно. Также в этой категории оказалось значительное количество пропусков. Помимо пропусков в колонке `общий трудовой стаж` обнаружились аномально высокие значения, исправить которые без дополнительной информации не представляется возможным. Такие низкие количественные характеристики искажают общий вывод исследования. В дополнении к существующим данным важную информацию могли бы предоставить сведения по размеру кредита, выданного заёмщику.